In [2]:
import pandas as pd
import numpy as np
from lxml import etree
import requests
import re
# 设置展示的最大宽度
desired_width = 3200
pd.set_option('display.width', desired_width)
# 显示的最大列数，解决中间被省略为...的问题
pd.set_option("display.max_columns",None)
# 将标题和数据左对齐,默认右对齐
pd.set_option('colheader_justify', 'left')

def get_data(addr_name):
    """获取指定url的数据, 并将结果新增city字段
       url:数据地址
       city:改地址的城市名称
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.1 \
    (KHTML, like Gecko) Chrome/14.0.835.163 Safari/535.1'}
    url = f'https://sh.lianjia.com/ershoufang/rs{addr_name}/'
    res = requests.get(url, headers=headers)
    # print(res.text)
    res_elements = etree.HTML(res.text)
    return res_elements


def get_house_num(res_elements):
    """获取小区在售的二手房套数"""
    table = res_elements.xpath('//h2[@class="total fl"]/span')
    ss = etree.tostring(table[0], encoding='utf-8').decode()
    num =re.findall('<span>(.+?)</span>.*',ss)[0].strip()
    return num

def get_house_info(res_elements):
    """获取在售的房子信息, 包括所属板块,几室几厅,面积, 售价"""
    table = res_elements.xpath('//div[@class="info clear"]')
    res = []
    print('-' * 60)
    for i,houss in enumerate(table):
        house_res = pd.Series()
        print(f'第{i+1}套')
        # 将选出来得条目转成text, 再转成网页树
        ss = etree.tostring(houss,encoding='utf-8').decode()
        houss = etree.HTML(ss)

        # 提取房子信息
        # 标题
        title = houss.xpath('//div[@class="title"]/a[@class="LOGCLICKDATA "]/text()')[0]
        print(f"title:{title}")
        house_res['title'] = title

        # 板块
        region_name = houss.xpath('//div[@class="positionInfo"]/a/text()')
        print(f"板块名称:{region_name}")
        house_res['板块名称'] = region_name[1]
        house_res['小区名称'] = region_name[0]

        # 房屋信息
        house_info = houss.xpath('//div[@class="houseInfo"]/text()')[0]
        print(f"房屋信息:{house_info}")
        house_res['室厅'] = house_info.split('|')[0]
        house_res['面积'] = house_info.split('|')[1]
        house_res['卧室朝向'] = house_info.split('|')[2]
        house_res['装修'] = house_info.split('|')[3]
        house_res['楼层'] = house_info.split('|')[4]
        house_res['年份'] = house_info.split('|')[5]
        house_res['类型'] = house_info.split('|')[6]

        # 价格
        price = houss.xpath('//div[@class="priceInfo"]/div[@class="totalPrice totalPrice2"]')[0].xpath('string(.)')
        print(f'总价:{price}')
        house_res['总价'] = price

        # 单价
        unit_price = houss.xpath('//div[@class="priceInfo"]/div[@class="unitPrice"]')[0].xpath('string(.)')
        print(f"单价:{unit_price}")
        house_res['单价'] = unit_price

        print('-'*60)
        res.append(house_res)
    return pd.DataFrame(res)


addr_name = '阳光苑民耀路97弄'
# 获取网页
res_elements = get_data(addr_name)

# 获取二手房套数
house_num = get_house_num(res_elements)
print(f'{addr_name} 共有{house_num}套二手房在售')

# 获取二手房的信息
res_df = get_house_info(res_elements)
res_df



阳光苑民耀路97弄 共有3套二手房在售
------------------------------------------------------------
第1套
title:此房户型方正，交通便利，小区位置好，视野开阔。
板块名称:['阳光苑民耀路97弄 ', '曹路']
房屋信息:2室1厅 | 74.07平米 | 南 | 简装 | 高楼层(共6层) | 1994年建 | 板楼
总价: 296万
单价:39,963元/平
------------------------------------------------------------
第2套
title:房子*，业主诚意出售，中间位置，视野采光好
板块名称:['阳光苑民耀路97弄 ', '曹路']
房屋信息:3室1厅 | 92.4平米 | 南 北 | 简装 | 高楼层(共6层) | 1999年建 | 板楼
总价: 364万
单价:39,394元/平
------------------------------------------------------------
第3套
title:房子满五w一，业主诚意出售，格局方正，视野采光好
板块名称:['阳光苑民耀路97弄 ', '曹路']
房屋信息:3室1厅 | 86平米 | 南 | 简装 | 高楼层(共6层) | 1994年建 | 板楼
总价: 340万
单价:39,535元/平
------------------------------------------------------------


<ipython-input-2-f4ab33b6d990>:41: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  house_res = pd.Series()


,title,板块名称,小区名称,室厅,面积,卧室朝向,装修,楼层,年份,类型,总价,单价
0,此房户型方正，交通便利，小区位置好，视野开阔。,曹路,阳光苑民耀路97弄,2室1厅,74.07平米,南,简装,高楼层(共6层),1994年建,板楼,296万,"39,963元/平"
1,房子*，业主诚意出售，中间位置，视野采光好,曹路,阳光苑民耀路97弄,3室1厅,92.4平米,南 北,简装,高楼层(共6层),1999年建,板楼,364万,"39,394元/平"
2,房子满五w一，业主诚意出售，格局方正，视野采光好,曹路,阳光苑民耀路97弄,3室1厅,86平米,南,简装,高楼层(共6层),1994年建,板楼,340万,"39,535元/平"
